In [5]:
!pip install duckdb-engine

  Using cached duckdb_engine-0.6.4-py3-none-any.whl (14 kB)
  Using cached numpy-1.23.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Using cached SQLAlchemy-1.4.44-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
  Using cached duckdb-0.6.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (14.4 MB)
  Using cached greenlet-2.0.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (544 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pythonwhat 2.23.1 requires asttokens~=1.1.10, but you have asttokens 2.0.8 which is incompatible.
pythonwhat 2.23.1 requires dill~=0.2.7.1, but you have dill 0.3.5.1 which is incompatible.
pythonwhat 2.23.1 requires jinja2~=2.10, but you have jinja2 3.1.2 which is incompatible.
--- Logging error ---
Traceback (most recent call last):
  Fil

In [6]:
import requests
import zipfile
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import duckdb
import sqlalchemy
from io import BytesIO

In [7]:
sslverify=True

url_root = "https://volby.cz/opendata/"
url_root_davky = "https://volby.cz/pls/"
volby = "prez2013"
opendatafile_home = "opendata.htm"
ciselniky = "PREZ2013ciselniky20130126_csv.zip"
registry = "PREZ2013reg20130126_csv.zip"
okrskova_data = "PREZ2013data20130126_csv.zip"
davky="vysledky_okrsky?davka="

url_opendata = url_root+volby+"/"+volby+"_"+opendatafile_home
url_davky = url_root_davky+volby+"/"+davky
url_ciselniky = url_root+volby+"/"+ciselniky
url_registry = url_root+volby+"/"+registry
url_okrskova_data = url_root+volby+"/"+okrskova_data

In [8]:
if not os.path.exists(volby):
    os.mkdir(volby)

In [11]:
def get_posledni_davka(url_davky):
    posledni_davka = requests.get(url_davky,verify=sslverify)
    root = ET.fromstring(posledni_davka.text)
    #for child in root:
    #    print (child.tag,child.attrib)
    davka = root[0].attrib
    davky_pocet = int(davka.get("PORADI_DAVKY")) + 1
    return(davky_pocet)

In [12]:
davky_pocet_kolo1 = get_posledni_davka(url_davky+"&kolo=1")


TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [13]:
davky_pocet_kolo2 = get_posledni_davka(url_davky+"&kolo=2")


TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [14]:
print(davky_pocet_kolo1,davky_pocet_kolo2)

NameError: name 'davky_pocet_kolo1' is not defined

In [43]:
def get_davky(kolo,davky_pocet):
    davkydir = os.path.join(volby,"davky"+kolo)
    dir_exists = os.path.exists(davkydir)
    if not dir_exists:
        os.mkdir(davkydir)
    for i in range(1,davky_pocet):
        url=url_davky+str(i)+"&kolo="+kolo
        response=requests.get(url,verify=sslverify)
        #print(response.status_code)
        if i<10:
            davkaurl=os.path.join(davkydir,"davka_0"+str(i)+".xml")                                
        else:
            davkaurl=os.path.join(davkydir,"davka_"+str(i)+".xml")
        with open(davkaurl,"wb") as file:
            file.write(response.content)

In [44]:
get_davky("1",davky_pocet_kolo1)


In [39]:
get_davky("2",davky_pocet_kolo2)

In [9]:
def get_cisregokr():
    for url in [url_ciselniky,url_registry,url_okrskova_data]:
        response = requests.get(url,verify=sslverify)
        zipfile.ZipFile(BytesIO(response.content)).extractall(path=volby)

In [10]:
get_cisregokr()

In [5]:
def zpracuj_davku(davkafile): #zpracování xml souboru dávky, vrací dict dávky, volebních dat okrsků, výsledků hlasování okrsků
    davka_list = []
    davky_list = []
    okrsek_list = []
    okrsky_list = []
    hlasy_list = []

    it = ET.iterparse(davkafile)
    for _, el in it:
        el.tag = el.tag.split('}', 1)[1]  # strip all namespaces
    root = it.root
    cas = list(root.items()[1])
    davka_list.append(cas)
    for element in root:
        if element.tag == "DAVKA":
            for davka_attr in element.items():
                davka_list.append(list(davka_attr))
            davky_list.append(dict(davka_list))
        elif element.tag == "OKRSEK":
            okrsek_dict = {}
            for okrsek_attr in element.items():
                okrsek_list.append(list(okrsek_attr))
                okrsek_dict.update(dict(okrsek_list))
            for okrsek_data in element:
                #print(okrsek_data.keys())
                if(okrsek_data.tag) == "UCAST_OKRSEK":
                    okrsek_dict.update(okrsek_data.attrib)
                else:
                    cis_obec=okrsek_dict["CIS_OBEC"]
                    cis_okrsek=okrsek_dict["CIS_OKRSEK"]
                    okrsek_hlasy=[]
                    for strana in okrsek_data.items():
                       okrsek_hlasy.append(strana[1])
                    hlasy_list.append([cis_obec,cis_okrsek,okrsek_hlasy[0],okrsek_hlasy[1]])
            okrsky_list.append(okrsek_dict)
    davka_df = pd.DataFrame(davky_list,index=["PORADI_DAVKY"])
    davka_okrsky_df = pd.DataFrame(okrsky_list)
    davka_hlasy_df = pd.DataFrame(hlasy_list,dtype='uint32',columns=["CIS_OBEC","CIS_OKRSEK","STRANA","HLASY"])

    return(davka_df,davka_okrsky_df,davka_hlasy_df)


In [ ]:
 
davky_dir = "davky"
davky = os.listdir(davky_dir)
davky.sort()
i=1
for davka_file in davky:
    if davka_file.endswith("xml"):
        xmldavka = os.path.join(davky_dir,davka_file)
        print(davka_file
              ,end='\r'
              )
        davka_df,okrsek_df,hlasy_davka_df = zpracuj_davku(xmldavka)
        if i == 1:
            davky_df = davka_df.astype({"DATUM_CAS_GENEROVANI":"datetime64"})
            #.astype()
            davky_okrsky_df = okrsek_df.astype({"DATUM_CAS_ZPRAC":"datetime64"})
            davky_hlasy_df = hlasy_davka_df
        else:
            davky_df = pd.concat([davky_df,davka_df]).astype({"DATUM_CAS_GENEROVANI":"datetime64"
            ,"PORADI_DAVKY":"uint8","OKRSKY_DAVKA":"uint16","OKRSKY_CELKEM":"uint16","OKRSKY_ZPRAC":"uint16"})
            davky_okrsky_df = pd.concat([davky_okrsky_df,okrsek_df]).astype({"DATUM_CAS_ZPRAC":"datetime64","CIS_OBEC":"uint32"
            ,"CIS_OKRSEK":"uint32","PORADI_ZPRAC":"uint16","OPAKOVANE":"bool","ZAPSANI_VOLICI":"uint32"
            ,"VYDANE_OBALKY":"uint32","ODEVZDANE_OBALKY":"uint32","PLATNE_HLASY":"uint32"})
            davky_hlasy_df = pd.concat([davky_hlasy_df,hlasy_davka_df]).astype("uint32")
            
        i=i+1
    


In [ ]:


# connect to an in-memory database
volby_davky = duckdb.connect()

# create the tables from the DataFrames
volby_davky.execute("CREATE TABLE davky AS SELECT * FROM davky_df")
volby_davky.execute("CREATE TABLE davky_okrsky AS SELECT * FROM davky_okrsky_df")
volby_davky.execute("CREATE TABLE davky_hlasy AS SELECT * FROM davky_hlasy_df")

# insert into the table "my_table" from the DataFrame "my_df"
#con.execute("INSERT INTO my_table SELECT * FROM my_df")

volby_davky.execute("DESCRIBE TABLE davky_hlasy").fetchall()

volby_davky.execute("SHOW TABLES").fetchall()

CSV_DIR = "csv_od"
csv_json_files = os.listdir(CSV_DIR)
csv_json_files.sort()
for file in csv_json_files:
    data,filetype = file.split(".")
    if file.endswith(".json"):
        df_name = data + "_df"
        with open(os.path.join(CSV_DIR,file), "r") as json_file:
            #df_name = pd.read_json(json_file)
            print("soubor:",file)
            json_data = json.load(json_file)
            for json_items in json_data.items():
                print(json_items)
            print()

            #s())
            #print(json_data.values())

In [ ]:
#test
cns_json = json.load(open('csv_od/cns.json'))
description = cns_json["dc:description"]
tableschema=cns_json["tableSchema"]
print(tableschema)
#print(tableschema["columns"])

cns_json_df = pd.DataFrame(tableschema["columns"])
cns_json_df

In [19]:
!python3 --version